In [1]:
%conda install -c conda-forge prophet

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: \ 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - conda-forge/noarch::aioitertools==0.7.1=pyhd8ed1ab_0
  - conda-forge/linux-64::importlib-metadata==3.7.0=py36h5fab9bb_0
  - conda-forge/noarch::typing-extensions==3.7.4.3=0
  - conda-forge/noarch::black==20.8b1=py_1
  - conda-forge/noarch::helpdev==0.7.1=pyhd8ed1ab_0
  - conda-forge/noarch::imageio==2.9.0=py_0
  - conda-forge/noarch::importlib_metadata==3.7.0=hd8ed1ab_0
  - conda-forge/linux-64::yarl==1.6.3=py36h8f6f2f9_1
  - conda-forge/noarch::flake8==3.8.4=py_0
  - conda-forge/noarch::jsonschema==3.2.0=py_2
  - conda-forge/linux-64::matplotlib-base==3.3.4=py36hd391965_0
  - conda-forge/linux-64::path==15.1.2=py36h5fab9bb_0
  - conda-forge/l

hdf4-4.2.15          | 950 KB    | ##################################### | 100% 
pylint-2.10.2        | 255 KB    | ##################################### | 100% 
hijri-converter-2.2. | 17 KB     | ##################################### | 100% 
convertdate-2.4.0    | 38 KB     | ##################################### | 100% 
notebook-6.3.0       | 6.3 MB    | ##################################### | 100% 
holidays-0.12        | 82 KB     | ##################################### | 100% 
cftime-1.5.0         | 226 KB    | ##################################### | 100% 
jupyterlab-3.2.8     | 5.8 MB    | ##################################### | 100% 
werkzeug-2.0.2       | 221 KB    | ##################################### | 100% 
pillow-8.2.0         | 688 KB    | ##################################### | 100% 
cmdstanpy-0.9.68     | 41 KB     | ##################################### | 100% 
astroid-2.7.3        | 330 KB    | ##################################### | 100% 
libnetcdf-4.7.4      | 1.3 M

In [2]:
import pandas as pd
from prophet import Prophet
from prophet.diagnostics import cross_validation
from prophet.diagnostics import performance_metrics
import itertools
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import glob

Matplotlib is building the font cache; this may take a moment.


# Read in Data and Prepare for Modeling 

In [3]:
england_df_list = list()

for fname in glob.glob("Data/Processed/Highways_England/*.csv"):
    print("Reading {}".format(fname))
    df = pd.read_csv(fname)
    df = df.sort_values(by="timestamp")
    england_df_list.append(df)

Reading Data/Processed/Highways_England/A47-6337-2_Eastbound_2019_Processed.csv
Reading Data/Processed/Highways_England/M6-7036B_Southbound_2019_Processed.csv
Reading Data/Processed/Highways_England/A64-9252-1_Northbound_2019_Processed.csv
Reading Data/Processed/Highways_England/M4-2156A_Westbound_2019_Processed.csv
Reading Data/Processed/Highways_England/M20-6552A2_Eastbound_2019_Processed.csv
Reading Data/Processed/Highways_England/M3-1537L_Eastbound_2019_Processed.csv
Reading Data/Processed/Highways_England/M18-7578A_Northbound_2019_Processed.csv
Reading Data/Processed/Highways_England/A590-9531-1_Eastbound_2019_Processed.csv
Reading Data/Processed/Highways_England/M25-4565A_Clockwise_2019_Processed.csv
Reading Data/Processed/Highways_England/A590-9634-1_Westbound_2019_Processed.csv
Reading Data/Processed/Highways_England/M4-3434B_Eastbound_2019_Processed.csv
Reading Data/Processed/Highways_England/M60-9327B_Counterclockwise_2019_Processed.csv
Reading Data/Processed/Highways_England

In [4]:
portland_df_list = list()

for fname in glob.glob("Data/Processed/Portland/*.csv"):
    print("Reading {}".format(fname))
    df = pd.read_csv(fname)
    df = df.sort_values(by="timestamp")
    portland_df_list.append(df)

Reading Data/Processed/Portland/OR217-100300_Southbound_2019_Processed.csv
Reading Data/Processed/Portland/SR 14-102003_Eastbound_2019_Processed.csv
Reading Data/Processed/Portland/I205-101068_Northbound_2019_Processed.csv
Reading Data/Processed/Portland/SR 14-102001_Westbound_2019_Processed.csv
Reading Data/Processed/Portland/OR217-100314_Northbound_2019_Processed.csv
Reading Data/Processed/Portland/I5-100688_Southbound_2019_Processed.csv
Reading Data/Processed/Portland/I84-101108_Eastbound_2019_Processed.csv
Reading Data/Processed/Portland/I84-101161_Westbound_2019_Processed.csv
Reading Data/Processed/Portland/R2 OR18-102111_Westbound_2019_Processed.csv
Reading Data/Processed/Portland/SR 500-1000104_Westbound_2019_Processed.csv
Reading Data/Processed/Portland/US26-100650_Westbound_2019_Processed.csv
Reading Data/Processed/Portland/US26-100627_Eastbound_2019_Processed.csv
Reading Data/Processed/Portland/R2 OR18-102113_Eastbound_2019_Processed.csv
Reading Data/Processed/Portland/I5-100

In [5]:
utah_df_list = list()

for fname in glob.glob("Data/Processed/Utah/*.csv"):
    print("Reading {}".format(fname))
    df = pd.read_csv(fname)
    df = df.sort_values(by="timestamp")
    utah_df_list.append(df)

Reading Data/Processed/Utah/I80-667_Westbound_2019_Processed.csv
Reading Data/Processed/Utah/I84-482_Westbound_2019_Processed.csv
Reading Data/Processed/Utah/I84-451_Eastbound_2019_Processed.csv
Reading Data/Processed/Utah/I215-31_Clockwise_2019_Processed.csv
Reading Data/Processed/Utah/US189-260_Westbound_2019_Processed.csv
Reading Data/Processed/Utah/I215-134_Counterclockwise_2019_Processed.csv
Reading Data/Processed/Utah/US40-634_Westbound_2019_Processed.csv
Reading Data/Processed/Utah/US6-3103115_Westbound_2019_Processed.csv
Reading Data/Processed/Utah/US89-483_Northbound_2019_Processed.csv
Reading Data/Processed/Utah/LegacyParkway-890_Southbound_2019_Processed.csv
Reading Data/Processed/Utah/I80-600_Eastbound_2019_Processed.csv
Reading Data/Processed/Utah/I15-3103178_Southbound_2019_Processed.csv
Reading Data/Processed/Utah/US40-635_Eastbound_2019_Processed.csv
Reading Data/Processed/Utah/LegacyParkway-810_Northbound_2019_Processed.csv
Reading Data/Processed/Utah/I15-749_Northboun

In [6]:
total_df_list = england_df_list + portland_df_list + utah_df_list

In [7]:
start_end = pd.read_csv("start_end_points.csv")
start_end["end"] = start_end["end"] + 1

In [8]:
start_end.head()

,start,end
0,11580,19644
1,17983,26047
2,14190,22254
3,24010,32074
4,3239,11303


In [9]:
subset_df_list = list()

In [10]:
for idx, df in enumerate(total_df_list):
        
    subset_df = df.iloc[start_end.iloc[idx,0]:start_end.iloc[idx,1], ]\
    .reset_index(drop=True).reset_index(drop=False)\
    .rename(columns={"index":"rn"})
    
    subset_df["train_val_test"] = np.where(subset_df["rn"]<(96*7*8),
                                           "train",
                                           np.where(subset_df["rn"]<(96*7*10),
                                                    "val",
                                                    "test"
                                                   )
                                       )
    
    subset_df_list.append(subset_df)

In [11]:
train_df_list = list()
test_df_list = list()

for df in subset_df_list:
    
    train_df = df.query("train_val_test != 'test'").copy()
    train_df = train_df[['timestamp', 'total_volume']]\
    .rename(columns={'timestamp':'ds', 'total_volume':'y'})
    train_df_list.append(train_df)
    
    test_df = df.query("train_val_test == 'test'").copy()
    test_df = test_df[['timestamp', 'total_volume']]\
    .rename(columns={'timestamp':'ds', 'total_volume':'y'})
    test_df_list.append(test_df)

# Fit Prophet Models via Cross Validation

In [13]:
param_grid = {  
    'changepoint_prior_scale': [0.001, 0.01, 0.1, 0.5],
    'seasonality_prior_scale': [0.01, 0.1, 1.0, 10.0],
}

In [14]:
all_params = [dict(zip(param_grid.keys(), v)) \
              for v in itertools.product(*param_grid.values())]

## England

In [ ]:
best_params = list()

In [86]:
for idx, train_df in enumerate(train_df_list[0:len(england_df_list)]):
    print(idx)
    rmses = [] 
    for params in all_params:
        m = Prophet(**params).fit(train_df)
        df_cv = cross_validation(m,  
                                 initial = '42 day', 
                                 period = '3.5 day', 
                                 horizon = '14 day',
                                 parallel = "processes")
        df_p = performance_metrics(df_cv, rolling_window=1)
        rmses.append(df_p['rmse'].values[0])
        
    best_param = all_params[np.argmin(rmses)]
    best_params.append(best_param)

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.


0


INFO:prophet:Making 4 forecasts with cutoffs between 2019-06-16 02:59:00 and 2019-06-26 14:59:00
INFO:prophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7f8ecf9c8eb8>
INFO:prophet:Skipping MAPE because y close to 0
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Making 4 forecasts with cutoffs between 2019-06-16 02:59:00 and 2019-06-26 14:59:00
INFO:prophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7f8ecfa05c18>
INFO:prophet:Skipping MAPE because y close to 0
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Making 4 forecasts with cutoffs between 2019-06-16 02:59:00 and 2019-06-26 14:59:00
INFO:prophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7f8f24d6e5f8>
INFO:prophet:Skipping MAPE because y close to 0
INFO:prophet:Disabling

1


INFO:prophet:Making 4 forecasts with cutoffs between 2019-08-21 19:44:00 and 2019-09-01 07:44:00
INFO:prophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7f8eea610978>
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Making 4 forecasts with cutoffs between 2019-08-21 19:44:00 and 2019-09-01 07:44:00
INFO:prophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7f8ee910be80>
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Making 4 forecasts with cutoffs between 2019-08-21 19:44:00 and 2019-09-01 07:44:00
INFO:prophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7f8ee9220f60>
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Making 4 forecasts with cutoffs between 2019-08-21 

...

INFO:prophet:Making 4 forecasts with cutoffs between 2019-04-11 16:44:00 and 2019-04-22 04:44:00
INFO:prophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7f8ef0729b70>
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Making 4 forecasts with cutoffs between 2019-04-11 16:44:00 and 2019-04-22 04:44:00
INFO:prophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7f8f24937048>
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Making 4 forecasts with cutoffs between 2019-04-11 16:44:00 and 2019-04-22 04:44:00
INFO:prophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7f8f24d90390>
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Making 4 forecasts with cutoffs between 2019-04-11 

36


INFO:prophet:Making 4 forecasts with cutoffs between 2019-05-10 04:59:00 and 2019-05-20 16:59:00
INFO:prophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7f8f24d90390>
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Making 4 forecasts with cutoffs between 2019-05-10 04:59:00 and 2019-05-20 16:59:00
INFO:prophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7f8f24f1e470>
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Making 4 forecasts with cutoffs between 2019-05-10 04:59:00 and 2019-05-20 16:59:00
INFO:prophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7f8f24f2b550>
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Making 4 forecasts with cutoffs between 2019-05-10 

37


INFO:prophet:Making 4 forecasts with cutoffs between 2019-10-31 14:29:00 and 2019-11-11 02:29:00
INFO:prophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7f8ee9215080>
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Making 4 forecasts with cutoffs between 2019-10-31 14:29:00 and 2019-11-11 02:29:00
INFO:prophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7f8f24939b38>
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Making 4 forecasts with cutoffs between 2019-10-31 14:29:00 and 2019-11-11 02:29:00
INFO:prophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7f8eea457dd8>
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Making 4 forecasts with cutoffs between 2019-10-31 

In [91]:
england_best_params = pd.DataFrame(best_params)

In [92]:
england_best_params.to_csv("england_best_params_prophet.csv", index=False)

## Portland

In [25]:
best_params = list()

In [ ]:
for idx, train_df in enumerate(train_df_list[len(england_df_list):(len(england_df_list)+len(portland_df_list))]):
    print(idx)
    rmses = [] 
    for params in all_params:
        m = Prophet(**params).fit(train_df)
        df_cv = cross_validation(m,  
                                 initial = '42 day', 
                                 period = '3.5 day', 
                                 horizon = '14 day',
                                 parallel = "processes")
        df_p = performance_metrics(df_cv, rolling_window=1)
        rmses.append(df_p['rmse'].values[0])
        
    best_param = all_params[np.argmin(rmses)]
    best_params.append(best_param)

0


INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Making 4 forecasts with cutoffs between 2019-04-24 06:59:00 and 2019-05-04 18:59:00
INFO:prophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7fd31659eb70>
INFO:prophet:Skipping MAPE because y close to 0
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Making 4 forecasts with cutoffs between 2019-04-24 06:59:00 and 2019-05-04 18:59:00
INFO:prophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7fd32dcfec88>
INFO:prophet:Skipping MAPE because y close to 0
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Making 4 forecasts with cutoffs between 2019-04-24 06:59:00 and 2019-05-04 18:59:00
INFO:prophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExe

1


INFO:prophet:Making 4 forecasts with cutoffs between 2019-03-01 11:59:00 and 2019-03-11 23:59:00
INFO:prophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7fd316648ac8>
INFO:prophet:Skipping MAPE because y close to 0
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Making 4 forecasts with cutoffs between 2019-03-01 11:59:00 and 2019-03-11 23:59:00
INFO:prophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7fd3166ebc88>
INFO:prophet:Skipping MAPE because y close to 0
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Making 4 forecasts with cutoffs between 2019-03-01 11:59:00 and 2019-03-11 23:59:00
INFO:prophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7fd3166de438>
INFO:prophet:Skipping MAPE because y close to 0
INFO:prophet:Disabling

2


...

INFO:prophet:Making 4 forecasts with cutoffs between 2019-05-16 15:14:00 and 2019-05-27 03:14:00
INFO:prophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7fd32dcfec18>
INFO:prophet:Skipping MAPE because y close to 0
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Making 4 forecasts with cutoffs between 2019-05-16 15:14:00 and 2019-05-27 03:14:00
INFO:prophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7fd316648b70>
INFO:prophet:Skipping MAPE because y close to 0
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Making 4 forecasts with cutoffs between 2019-05-16 15:14:00 and 2019-05-27 03:14:00
INFO:prophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7fd316648160>
INFO:prophet:Skipping MAPE because y close to 0
INFO:prophet:Disabling

17


INFO:prophet:Making 4 forecasts with cutoffs between 2019-09-16 10:59:00 and 2019-09-26 22:59:00
INFO:prophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7fd3166de320>
INFO:prophet:Skipping MAPE because y close to 0
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Making 4 forecasts with cutoffs between 2019-09-16 10:59:00 and 2019-09-26 22:59:00
INFO:prophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7fd316648ac8>
INFO:prophet:Skipping MAPE because y close to 0
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Making 4 forecasts with cutoffs between 2019-09-16 10:59:00 and 2019-09-26 22:59:00
INFO:prophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7fd316648748>
INFO:prophet:Skipping MAPE because y close to 0
INFO:prophet:Disabling

18


INFO:prophet:Making 4 forecasts with cutoffs between 2019-07-26 01:59:00 and 2019-08-05 13:59:00
INFO:prophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7fd3166eb5f8>
INFO:prophet:Skipping MAPE because y close to 0
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Making 4 forecasts with cutoffs between 2019-07-26 01:59:00 and 2019-08-05 13:59:00
INFO:prophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7fd316493a20>
INFO:prophet:Skipping MAPE because y close to 0
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Making 4 forecasts with cutoffs between 2019-07-26 01:59:00 and 2019-08-05 13:59:00
INFO:prophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7fd31659e390>
INFO:prophet:Skipping MAPE because y close to 0
INFO:prophet:Disabling

In [35]:
portland_best_params = pd.DataFrame(best_params)

In [36]:
portland_best_params.to_csv("portland_best_params_prophet.csv", index=False)

## Utah

In [37]:
best_params = list()

In [38]:
for idx, train_df in enumerate(train_df_list[(len(england_df_list)+len(portland_df_list)):len(train_df_list)]):
    print(idx)
    rmses = [] 
    for params in all_params:
        m = Prophet(**params).fit(train_df)
        df_cv = cross_validation(m,  
                                 initial = '42 day', 
                                 period = '3.5 day', 
                                 horizon = '14 day',
                                 parallel = "processes")
        df_p = performance_metrics(df_cv, rolling_window=1)
        rmses.append(df_p['rmse'].values[0])
        
    best_param = all_params[np.argmin(rmses)]
    best_params.append(best_param)

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.


0


INFO:prophet:Making 4 forecasts with cutoffs between 2019-05-16 12:59:00 and 2019-05-27 00:59:00
INFO:prophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7fd3166eb5f8>
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Making 4 forecasts with cutoffs between 2019-05-16 12:59:00 and 2019-05-27 00:59:00
INFO:prophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7fd3164930b8>
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Making 4 forecasts with cutoffs between 2019-05-16 12:59:00 and 2019-05-27 00:59:00
INFO:prophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7fd30efe2cf8>
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Making 4 forecasts with cutoffs between 2019-05-16 

1


INFO:prophet:Making 4 forecasts with cutoffs between 2019-05-10 02:59:00 and 2019-05-20 14:59:00
INFO:prophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7fd37d83b4a8>
INFO:prophet:Skipping MAPE because y close to 0
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Making 4 forecasts with cutoffs between 2019-05-10 02:59:00 and 2019-05-20 14:59:00
INFO:prophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7fd32dcfecf8>
INFO:prophet:Skipping MAPE because y close to 0
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Making 4 forecasts with cutoffs between 2019-05-10 02:59:00 and 2019-05-20 14:59:00
INFO:prophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7fd30efc1be0>
INFO:prophet:Skipping MAPE because y close to 0
INFO:prophet:Disabling

...

INFO:prophet:Making 4 forecasts with cutoffs between 2019-09-20 04:44:00 and 2019-09-30 16:44:00
INFO:prophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7fd37d83b4a8>
INFO:prophet:Skipping MAPE because y close to 0
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Making 4 forecasts with cutoffs between 2019-09-20 04:44:00 and 2019-09-30 16:44:00
INFO:prophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7fd37d83b4a8>
INFO:prophet:Skipping MAPE because y close to 0
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Making 4 forecasts with cutoffs between 2019-09-20 04:44:00 and 2019-09-30 16:44:00
INFO:prophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7fd3169a94e0>
INFO:prophet:Skipping MAPE because y close to 0
INFO:prophet:Disabling

17


INFO:prophet:Making 4 forecasts with cutoffs between 2019-03-02 15:29:00 and 2019-03-13 03:29:00
INFO:prophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7fd30f051f60>
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Making 4 forecasts with cutoffs between 2019-03-02 15:29:00 and 2019-03-13 03:29:00
INFO:prophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7fd30f051f60>
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Making 4 forecasts with cutoffs between 2019-03-02 15:29:00 and 2019-03-13 03:29:00
INFO:prophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7fd37d83b4a8>
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Making 4 forecasts with cutoffs between 2019-03-02 

18


INFO:prophet:Making 4 forecasts with cutoffs between 2019-06-20 17:14:00 and 2019-07-01 05:14:00
INFO:prophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7fd316661518>
INFO:prophet:Skipping MAPE because y close to 0
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Making 4 forecasts with cutoffs between 2019-06-20 17:14:00 and 2019-07-01 05:14:00
INFO:prophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7fd3166ebfd0>
INFO:prophet:Skipping MAPE because y close to 0
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Making 4 forecasts with cutoffs between 2019-06-20 17:14:00 and 2019-07-01 05:14:00
INFO:prophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7fd32ef3a9b0>
INFO:prophet:Skipping MAPE because y close to 0
INFO:prophet:Disabling

In [39]:
utah_best_params = pd.DataFrame(best_params)

In [40]:
utah_best_params.to_csv("utah_best_params_prophet.csv", index=False)

# Forecast with Prophet Models Found Above

In [12]:
england_params = pd.read_csv("england_best_params_prophet.csv")
portland_params = pd.read_csv("portland_best_params_prophet.csv")
utah_params = pd.read_csv("utah_best_params_prophet.csv")
total_params = england_params.append(portland_params).append(utah_params)

In [13]:
rmse_1_week_list = list()
rmse_2_week_list = list()
mae_1_week_list = list()
mae_2_week_list = list()
forecast_dict = dict()

for i in range(len(train_df_list)):
    model_params = total_params.iloc[i,].to_dict()
    mod = Prophet(**model_params)
    mod.fit(train_df_list[i])
    forecasts = mod.predict(test_df_list[i])
    
    rmse_1_week = mean_squared_error(test_df_list[i].y[0:(len(test_df_list[i].y)//2)], 
                                     forecasts.yhat[0:(len(test_df_list[i].y)//2)], 
                                     squared=False)
    mae_1_week = mean_absolute_error(test_df_list[i].y[0:(len(test_df_list[i].y)//2)], 
                                     forecasts.yhat[0:(len(test_df_list[i].y)//2)])
    rmse_2_week = mean_squared_error(test_df_list[i].y, 
                                     forecasts.yhat, 
                                     squared=False)
    mae_2_week = mean_absolute_error(test_df_list[i].y,
                                     forecasts.yhat)
    
    forecast_dict[i] = forecasts
    rmse_1_week_list.append(rmse_1_week)
    mae_1_week_list.append(mae_1_week)
    rmse_2_week_list.append(rmse_2_week)
    mae_2_week_list.append(mae_2_week)

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=Tru

In [14]:
np.mean(rmse_1_week_list)

73.18685708646198

In [15]:
np.mean(rmse_2_week_list)

76.31010562623032

In [16]:
np.mean(mae_1_week_list)

53.36211953146875

In [17]:
np.mean(mae_2_week_list)

55.65218137070171

In [18]:
prophet_results = pd.DataFrame(list(zip(rmse_1_week_list, rmse_2_week_list, mae_1_week_list, mae_2_week_list)),
                               columns=["rmse_1_week", "rmse_2_week", "mae_1_week", "mae_2_week"])

In [19]:
prophet_results.head()

,rmse_1_week,rmse_2_week,mae_1_week,mae_2_week
0,43.870212,42.743961,31.819951,30.679881
1,223.778139,208.317425,152.593097,145.390576
2,86.695002,84.719545,64.585091,63.717194
3,86.435739,82.804374,64.056908,60.708471
4,84.988284,88.992394,61.242115,65.032507


In [20]:
prophet_results.to_csv("prophet_performance.csv", index=False)

In [30]:
forecasts_df = pd.DataFrame()

In [31]:
for i in range(len(forecast_dict)):
    forecast_df_sub = forecast_dict[i]
    forecast_df_sub["ts_index"] = i
    forecasts_df = forecasts_df.append(forecast_df_sub)

In [35]:
forecasts_df.to_csv("raw_prophet_forecasts.csv")